In [3]:
!pip install --upgrade setuptools pip

  Using cached setuptools-72.1.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached setuptools-72.1.0-py3-none-any.whl (2.3 MB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [2]:
pip install scikit-learn

In [3]:
pip install pandas

In [2]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# Download dataset
news_data = fetch_20newsgroups(subset='all', categories=['sci.space', 'comp.graphics', 'rec.autos', 'talk.politics.misc'], remove=('headers', 'footers', 'quotes'))

# Convert to DataFrame
df = pd.DataFrame({'text': news_data.data, 'category': news_data.target})

# Save to CSV
df.to_csv('news_articles.csv', index=False)


In [3]:
# Load the dataset
df = pd.read_csv('news_articles.csv')
print(df.head())

                                                text  category
0  Forwarded from Doug Griffith, Magellan Project...         2
1  \nYour starting to sound like a little child w...         3
2  I am trying to find out anything I can about a...         0
3  \nat least be honest.  velcro (tm) dates from ...         2
4  \nDon't get too excited; Signetics, not Motoro...         0


In [7]:
print(df)
print(type(df))

                                                   text  category
0     Forwarded from Doug Griffith, Magellan Project...         2
1     \nYour starting to sound like a little child w...         3
2     I am trying to find out anything I can about a...         0
3     \nat least be honest.  velcro (tm) dates from ...         2
4     \nDon't get too excited; Signetics, not Motoro...         0
...                                                 ...       ...
3720  # #The argument that "slavery was a dying inst...         3
3721  \nBy '8 grey level images' you mean 8 items of...         0
3722   \n\nFinding the circumcenter of a tetrahedron...         0
3723  I agree with Gaia. Even though the Saturn has ...         1
3724  1) Lucas film\n2) Pixar\n3) 3D/Eye Inc.\n4) Li...         0

[3725 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>


MODEL 1 - GPT-Neo Fine Tunninprig

In [6]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 40.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.10

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments
import pandas as pd
import torch
from torch.utils.data import Dataset

# Load dataset
df = pd.read_csv('news_articles.csv')

# Convert the text column to a list of strings
texts = df['text'].astype(str).tolist()

# Load pre-trained model and tokenizer
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-1.3B')
tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')

# Set pad_token as eos_token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the list of strings
tokenized_datasets = tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

# Create a custom Dataset class
class NewsDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids']
        return item

# Create dataset object
news_dataset = NewsDataset(tokenized_datasets)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=news_dataset,
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained('./gpt_neo_finetuned')
tokenizer.save_pretrained('./gpt_neo_finetuned')


<ipython-input-12-7db4f1ac9797>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


MODEL 2 - BERT Fine Tunning

In [4]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [7]:
pip install transformers

In [11]:
!pip uninstall torch -y

Found existing installation: torch 2.3.1+cu121
Uninstalling torch-2.3.1+cu121:
  Successfully uninstalled torch-2.3.1+cu121


In [12]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.4.0 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.4.0 which is incompatible.


In [7]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# Download dataset
news_data = fetch_20newsgroups(subset='all', categories=['sci.space', 'comp.graphics', 'rec.autos', 'talk.politics.misc'], remove=('headers', 'footers', 'quotes'))

# Convert to DataFrame
df = pd.DataFrame({'text': news_data.data, 'label': news_data.target})

# Save to CSV
df.to_csv('news_articles.csv', index=False)

In [8]:
# Load the dataset
df = pd.read_csv('news_articles.csv')
print(df.head())

                                                text  label
0  Forwarded from Doug Griffith, Magellan Project...      2
1  \nYour starting to sound like a little child w...      3
2  I am trying to find out anything I can about a...      0
3  \nat least be honest.  velcro (tm) dates from ...      2
4  \nDon't get too excited; Signetics, not Motoro...      0


In [9]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import torch
from torch.utils.data import Dataset

# Load dataset
df = pd.read_csv('news_articles.csv')

# Preprocess data: Convert text and labels to appropriate formats
texts = df['text'].astype(str).tolist()
labels = df['label'].tolist()

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(set(labels)))

# Tokenize dataset
tokenized_datasets = tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

# Create a custom Dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create dataset object
dataset = TextClassificationDataset(tokenized_datasets, labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained('./bert_finetuned')
tokenizer.save_pretrained('./bert_finetuned')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-7c492f66ed98>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

T5 Fine Tunning

In [14]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# Download dataset
news_data = fetch_20newsgroups(subset='all', categories=['sci.space', 'comp.graphics', 'rec.autos', 'talk.politics.misc'], remove=('headers', 'footers', 'quotes'))

# Convert to DataFrame
df = pd.DataFrame({'input_text': news_data.data, 'target_text': news_data.target})

# Save to CSV
df.to_csv('text_to_text_data.csv', index=False)

In [15]:
# Load the dataset
df = pd.read_csv('text_to_text_data.csv')
print(df.head())

                                          input_text  target_text
0  Forwarded from Doug Griffith, Magellan Project...            2
1  \nYour starting to sound like a little child w...            3
2  I am trying to find out anything I can about a...            0
3  \nat least be honest.  velcro (tm) dates from ...            2
4  \nDon't get too excited; Signetics, not Motoro...            0


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd
import torch
from torch.utils.data import Dataset

# Load dataset
df = pd.read_csv('text_to_text_data.csv')

# Preprocess data: Convert input and target texts to lists
inputs = df['input_text'].astype(str).tolist()
targets = df['target_text'].astype(str).tolist()

# Load pre-trained T5 model and tokenizer
model_name = 't5-small'  # You can also use 't5-base' or 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize inputs and targets
class T5Dataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_length=512):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        labels = target_encoding["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": labels.flatten()
        }

# Create dataset object
dataset = T5Dataset(inputs, targets, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="epoch",
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./t5_finetuned')
tokenizer.save_pretrained('./t5_finetuned')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


MODAL EVALUATION:
1)Load Model
2)Evaluate model

1)LOAD MODEL

In [ ]:
from transformers import pipeline

# Load models
gpt_neo = pipeline('text-generation', model='./gpt_neo_finetuned')
bert = pipeline('text-classification', model='./bert_finetuned')
t5 = pipeline('text2text-generation', model='./t5_finetuned')


2)EVALUATION MODEL

In [ ]:
sample_input = "The future of artificial intelligence is"

# Generate articles
gpt_neo_output = gpt_neo(sample_input, max_length=100, num_return_sequences=1)
t5_output = t5(sample_input, max_length=100, num_return_sequences=1)

# Print results
print("GPT-Neo Output:", gpt_neo_output[0]['generated_text'])
print("T5 Output:", t5_output[0]['generated_text'])


In [ ]:
#CALCULATE METRICS
from sklearn.metrics import classification_report

# Assuming you have true labels and predicted labels
y_true = df['category']
y_pred = [model.predict(text) for text in df['text']]

print(classification_report(y_true, y_pred))
